### Demo Spark 2

In this demo, let's find all the movies with the lowest average rating, but with DataFrames this time.  

In the Data section of Databricks, create a table with the `u.item` file. 
Name the first 3 colums as the following: `movie_id`, `title`, `release_date`. Name the table `u_item`.

Create another table with the `u.data` file. 
Name the colums as the following: `user_id`, `movie_id`, `rating`, `timestamp`. Name the table `u_data`.


Refer to the page "Add a table in Databricks" in Module 6 in Canvas for more details on how to create a table in Databricks.

In [2]:
#List the path of the u.item and u.data

display(dbutils.fs.ls("/FileStore/tables/"))

In [3]:
#Create the dataframes from the table created in Databricks. 

dataDF = sqlContext.sql("SELECT user_id, movie_id, rating FROM u_data")

itemDF = sqlContext.sql("SELECT movie_id, title FROM u_item")


In [4]:
#Compute average rating for each movie
#movieDF has the form: (movie_id, rating)
movieDF = dataDF.select("movie_id", "rating").cache()

#averageRatings has the form: (movie_id, avg(rating)) 
#where avg(rating) is the average rating by movie_id
averageRatings = movieDF.groupBy("movie_id").avg("rating")

# Compute counts of ratings for each movie ID
#counts has the form: (movie_id, count)
counts = movieDF.groupBy("movie_id").count() 


In [5]:
# Join averageRatings and counts with movie_id
averageAndCounts = counts.join(averageRatings, "movie_id").join(itemDF, "movie_id")
#averageAndCounts.show()

# Pull the top ten results
topTen = averageAndCounts.orderBy("avg(rating)").select("title", "avg(rating)", "count").show(10)